# Polynomial Optimization

**Contributed by**: Benoît Legat

## Introduction

Consider the polynomial optimization problem [L09, Example 2.2] of
minimizing the polynomial $x^3 - x^2 + 2xy - y^2 + y^3$
over the polyhedron defined by the inequalities $x \ge 0, y \ge 0$ and $x + y \geq 1$.

[L09] Lasserre, J. B.
*Moments, positive polynomials and their applications*.
World Scientific, **2009**.

In [1]:
using DynamicPolynomials
@polyvar x y
p = x^3 - x^2 + 2x*y -y^2 + y^3
using SumOfSquares
S = @set x >= 0 && y >= 0 && x + y >= 1
p(x=>1, y=>0), p(x=>1//2, y=>1//2), p(x=>0, y=>1)

(0, 1//4, 0)

The optimal solutions are $(x, y) = (1, 0)$ and $(x, y) = (0, 1)$ with objective value $0$ but [Ipopt](https://github.com/jump-dev/Ipopt.jl/) only finds the local minimum $(1/2, 1/2)$ with objective value $1/4$.

In [2]:
import Ipopt
model = Model(optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0))
@variable(model, a >= 0)
@variable(model, b >= 0)
@constraint(model, a + b >= 1)
@NLobjective(model, Min, a^3 - a^2 + 2a*b - b^2 + b^3)
optimize!(model)
@show termination_status(model)
@show value(a)
@show value(b)
@show objective_value(model)

Iter: 12 Ap: 9.00e-01 Pobj: -6.0000000e+00 Ad: 1.00e+00 Dobj: -6.0000000e+00 
Success: SDP solved
Primal objective value: -6.0000000e+00 
Dual objective value: -6.0000000e+00 
Relative primal infeasibility: 8.19e-12 
Relative dual infeasibility: 3.44e-11 
Real Relative Gap: 3.74e-10 
XZ Relative Gap: 7.09e-10 
DIMACS error measures: 8.95e-12 0.00e+00 5.84e-11 0.00e+00 3.74e-10 7.09e-10

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

termination_status(model) = MathOptInterface.LOCALLY_SOLVED
value(a) = 0.4999999966705879
value(b) = 0.49999999667061656
objective_value(model) = 0.24999999500590336


0.24999999500590336

Note that the problem can be written equivalently as follows using [registered functions](https://jump.dev/JuMP.jl/stable/manual/nlp/#Register-a-function).

In [3]:
using Ipopt
model = Model(optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0))
@variable(model, a >= 0)
@variable(model, b >= 0)
@constraint(model, a + b >= 1)
peval(a, b) = p(x=>a, y=>b)
register(model, :peval, 2, peval, autodiff=true)
@NLobjective(model, Min, peval(a, b))
optimize!(model)
@show termination_status(model)
@show value(a)
@show value(b)
@show objective_value(model)

# Sum-of-Squares approach

termination_status(model) = MathOptInterface.LOCALLY_SOLVED
value(a) = 0.499999995002878
value(b) = 0.4999999950028773
objective_value(model) = 0.24999999250431656


0.24999999250431656

We will now see how to find the optimal solution using Sum of Squares Programming.
We first need to pick an SDP solver, see [here](https://jump.dev/JuMP.jl/v1.8/installation/#Supported-solvers) for a list of the available choices.

In [4]:
import CSDP
solver = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true)

MathOptInterface.OptimizerWithAttributes(CSDP.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.Silent() => true])

A Sum-of-Squares certificate that $p \ge \alpha$ over the domain `S`, ensures that $\alpha$ is a lower bound to the polynomial optimization problem.
The following program searches for the largest lower bound and finds zero.

In [5]:
model = SOSModel(solver)
@variable(model, α)
@objective(model, Max, α)
@constraint(model, c3, p >= α, domain = S)
optimize!(model)
@show termination_status(model)
@show objective_value(model)

termination_status(model) = MathOptInterface.OPTIMAL
objective_value(model) = -1.7799189899747603e-10


-1.7799189899747603e-10

Using the solution $(1/2, 1/2)$ found by Ipopt of objective value $1/4$
and this certificate of lower bound $0$ we know that the optimal objective value is in the interval $[0, 1/4]$
but we still do not know what it is (if we consider that we did not try the solutions $(1, 0)$ and $(0, 1)$ as done in the introduction).
If the dual of the constraint `c3` was atomic, its atoms would have given optimal solutions of objective value $0$ but that is not the case.

In [6]:
ν3 = moment_matrix(c3)
extractatoms(ν3, 1e-3) # Returns nothing as the dual is not atomic

Fortunately, there is a hierarchy of programs with increasingly better bounds that can be solved until we get one with atom dual variables.
This comes from the way the Sum-of-Squares constraint with domain `S` is formulated.
The polynomial $p - \alpha$ is guaranteed to be nonnegative over the domain `S` if there exists Sum-of-Squares polynomials $s_0$, $s_1$, $s_2$, $s_3$ such that
$$
p - \alpha = s_0 + s_1 x + s_2 y + s_3 (x + y - 1).
$$
Indeed, in the domain `S`, $x$, $y$ and $x + y - 1$ are nonnegative so the right-hand side is a sum of squares hence is nonnegative.
Once the degrees of $s_1$, $s_2$ and $s_3$ have been decided, the degree needed for $s_0$ will be determined but we have a freedom in choosing the degrees of $s_1$, $s_2$ and $s_3$.
By default, they are chosen so that the degrees of $s_1 x$, $s_2 y$ and $s_3 (x + y - 1)$ match those of $p - \alpha$ but this can be overwritten using the `maxdegree` keyword argument.

### The maxdegree keyword argument

The maximum total degree (i.e. maximum sum of the exponents of $x$ and $y$) of the monomials of $p$ is 3 so the constraint in the program above is equivalent to `@constraint(model, p >= α, domain = S, maxdegree = 3)`.
That is, since $x$, $y$ and $x + y - 1$ have total degree 1, the sum of squares polynomials $s_1$, $s_2$ and $s_3$ have been chosen with maximum total degree $2$.
Since these polynomials are sums of squares, their degree must be even so the next maximum total degree to try is 4.
For this reason, the keywords `maxdegree = 4` and `maxdegree = 5` have the same effect in this example.
In general, if the polynomials in the domain are not all odd or all even, each value of `maxdegree` has a different effect in the choice of the maximum total degree of some $s_i$.

In [7]:
model = SOSModel(solver)
@variable(model, α)
@objective(model, Max, α)
@constraint(model, c5, p >= α, domain = S, maxdegree = 5)
optimize!(model)
@show termination_status(model)
@show objective_value(model)

termination_status(model) = MathOptInterface.OPTIMAL
objective_value(model) = -3.4520638569901507e-9


-3.4520638569901507e-9

As shown below, for `maxdegree = 5`, the dual variable is atomic as it is the moments of the measure
$$0.5 \delta(x-1, y) + 0.5 \delta(x, y-1)$$
where $\delta(x, y)$ is the dirac measure centered at $(0, 0)$.
Therefore the program provides both a certificate that $0$ is a lower bound and a certificate that it is also an upper bound since it is attained at the global minimizers $(1, 0)$ and $(0, 1)$.

In [8]:
ν5 = moment_matrix(c5)
extractatoms(ν5, 1e-3)

Atomic measure on the variables x, y with 2 atoms:
 at [-0.00027755883462882397, 1.0002775595143842] with weight 0.4999650862222643
 at [0.9997947978356583, 0.00020520284409647257] with weight 0.5005925828244098

## A deeper look into atom extraction

The `extractatoms` function requires a `ranktol` argument that we have set to `1e-3` in the preceding section.
This argument is used to transform the dual variable into a system of polynomials equations whose solutions give the atoms.
This transformation uses the SVD decomposition of the moment matrix and discards the equations corresponding to a singular value lower than `ranktol`.
When this system of equation has an infinite number of solutions, `extractatoms` concludes that the measure is not atomic.
For instance, with `maxdegree = 3`, we obtain the system
$$x + y = 1$$
which contains a whole line of solution.
This explains `extractatoms` returned `nothing`.

In [9]:
ν3 = moment_matrix(c3)
SumOfSquares.MultivariateMoments.computesupport!(ν3, 1e-3)

Algebraic Set defined by 1 equalitty
 -x - 1.0000000000000002*y + 1.00000000004984 = 0


With `maxdegree = 5`, we obtain the system
$$
\begin{aligned}
  x + y & = 1\\
  y^2 & = y\\
  xy & = 0\\
  x^2 + y & = 1
\end{aligned}
$$

In [10]:
ν5 = moment_matrix(c5)
SumOfSquares.MultivariateMoments.computesupport!(ν5, 1e-3)

Algebraic Set defined by 3 equalities
 -x - 0.9999999999999994*y + 1.0000000006797547 = 0
 -x*y + 0.44029103700223565*y^2 - 0.44098635462040214*y + 0.00029563385033835106 = 0
 -x^2 + 1.0000005296698848*y^2 - 2.003158975989828*y + 1.0015792234970937 = 0


This system can be reduced to the equivalent system
$$
\begin{aligned}
  x + y & = 1\\
  y^2 & = y
\end{aligned}
$$
which has the solutions $(0, 1)$ and $(1, 0)$.

In [11]:
SemialgebraicSets.computegröbnerbasis!(ideal(ν5.support))
ν5.support

Algebraic Set defined by 2 equalities
 x + 0.9999999999999994*y - 1.0000000006797547 = 0
 y^2 - 1.0004827623584807*y + 0.00020525980009822995 = 0


The function `extractatoms` then reuses the matrix of moments to find the weights $1/2$, $1/2$ corresponding to the diracs centered respectively at $(0, 1)$ and $(1, 0)$.
This details how the function obtained the result
$$0.5 \delta(x-1, y) + 0.5 \delta(x, y-1)$$
given in the previous section.

## HomotopyContinuation

As discussed in the previous section, the atom extraction relies on the solution
of a system of algebraic equations. The `extractatoms` function takes an optional
`solver` argument that is used to solve this system of equation.
If no solver is provided, the default solver of SemialgebraicSets.jl is used which
currently computes the Gröbner basis, then the multiplication matrices and
then the Schur decomposition of a random combination of these matrices.
As the system of equations is obtained from a numerical solution and is represented
using floating point coefficients, homotopy continuation is recommended as it is
more numerically robust than Gröbner basis computation.
The following uses homotopy continuation to solve the system of equations.

In [12]:
using HomotopyContinuation
solver = SemialgebraicSetsHCSolver(; excess_residual_tol = 2e-2, real_tol = 2e-2, compile = false)
extractatoms(ν5, 1e-3, solver)

Tracking 4 paths...  50%|███████████████▌               |  ETA: 0:00:13
  # paths tracked:                  2
  # non-singular solutions (real):  0 (0)
  # singular endpoints (real):      0 (0)




Tracking 4 paths... 100%|███████████████████████████████| Time: 0:00:13
  # paths tracked:                  4
  # non-singular solutions (real):  0 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         0 (0)


Atomic measure on the variables x, y with 2 atoms:
 at [1.000685059977968, -2.9889983359122304e-6] with weight 0.4995588792459913
 at [0.0006018567067685059, 1.0000443425236136] with weight 0.5003877158802152

As the system has 3 equations for 2 variables and the coefficients of the equations
are to be treated with tolerance since they originate from the solution of an SDP,
we need to set `excess_residual_tol` and `real_tol` to a high tolerance otherwise,
HomotopyContinuation would consider that there is no solution.
Indeed, as the system is overdetermined (it has more equations than variables)
HomotopyContinuation expects to have excess solution hence it filters out
excess solution among the solution found. It determines which solution are in excess
by comparing the infinity norm of the residuals of the equations at the solution with `excess_residual_tol`.
It also filters out solution for which the absolute value of the imaginary part of one of the entry
is larger than `real_tol` and strips out the imaginary part.
The raw solutions obtained by HomotopyContinuation can be obtained as follows:

In [13]:
F = HomotopyContinuation.System(ν5.support)
res = HomotopyContinuation.solve(F, solver.options...)
path_results(res)

Tracking 4 paths...  50%|███████████████▌               |  ETA: 0:00:03
  # paths tracked:                  2
  # non-singular solutions (real):  0 (0)
  # singular endpoints (real):      0 (0)




Tracking 4 paths... 100%|███████████████████████████████| Time: 0:00:03
  # paths tracked:                  4
  # non-singular solutions (real):  0 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         0 (0)


4-element Vector{HomotopyContinuation.PathResult}:
 PathResult:
 • return_code → :excess_solution
 • solution → ComplexF64[-0.21888392402274462 - 0.16053278354251455im, 0.6063922192302618 + 0.6613414143208797im]
 • accuracy → 0.79117
 • residual → 0.4296
 • condition_jacobian → 6.5486
 • steps → 22 / 0
 • extended_precision → false
 • path_number → 1

 PathResult:
 • return_code → :excess_solution
 • solution → ComplexF64[1.0007493815844064 - 0.00047916089231632474im, -3.0436118829412792e-5 + 0.00015441009219415412im]
 • accuracy → 0.00078889
 • residual → 0.00057799
 • condition_jacobian → 66.07
 • steps → 41 / 0
 • extended_precision → false
 • path_number → 2

 PathResult:
 • return_code → :excess_solution
 • solution → ComplexF64[-2.560177564824281 + 0.2713566679757274im, -1.7348350516212363 + 1.095253497092441im]
 • accuracy → 5.4685
 • residual → 1.9575
 • condition_jacobian → 11.737
 • steps → 35 / 0
 • extended_precision → false
 • path_number → 3

 PathResult:
 • return_code →

The printed `residual` above shows why `2e-2` allows to filter how the 2 actual
solutions from the 2 excess solutions.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*